In [1]:
from bs4 import BeautifulSoup
import requests
import csv
import pandas as pd
print (" Librabries imported")

 Librabries imported


In [79]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source,'lxml')

table = soup.find('table',class_='wikitable sortable')
A=[]
B=[]
C=[]


for row in table.findAll('tr'):
    cells=row.findAll('td')
    if len(cells)==3:
        A.append(cells[0].find(text=True))
        B.append(cells[1].find(text=True))
        C.append(cells[2].find(text=True))
df=pd.DataFrame(A,columns=['Postcode'])
df['Borough']=B
df['Neighbourhood']=C
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [3]:
df.shape

(288, 3)

In [81]:
#Ignore cells with a borough that is Not assigned
df.drop(df[df['Borough'] == "Not assigned"].index,axis=0,inplace=True)
df.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


In [86]:
# Combine same postal codes by moving neighbouhoods in same row; separated by commas

df2=df.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(', '.join).reset_index()
df2['Neighbourhood']=df2['Neighbourhood'].replace(to_replace='\n', value='', regex=True)
df2.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [87]:
# look for unassigned neighbourhoods postcodes
df2.query('Neighbourhood == "Not assigned"')

,Postcode,Borough,Neighbourhood
85,M7A,Queen's Park,Not assigned


In [95]:
# replacing neighbourhoods postcodes with borough
df2.loc[df2.Neighbourhood == 'Not assigned', "Neighbourhood"] = df2.Borough
df2.query('Postcode == "M7A"')

,Postcode,Borough,Neighbourhood
85,M7A,Queen's Park,Queen's Park


In [97]:
# number of rows of your dataframe
df2.shape

(103, 3)